In [ ]:
import os
import json
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.metrics import AUC, Precision, Recall

In [ ]:
# Data Path and Configurations
dataset_dir = r"/dataset"
save_path = r'/vgg16_results.json'

In [ ]:
# Initialize results by loading existing data if available
def load_existing_results(path):
    try:
        with open(path, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return []

# Load results
all_results = load_existing_results(save_path)

# Adjusted save_results function
def save_results(metrics):
    # Load existing results if any
    try:
        with open(save_path, 'r') as f:
            all_results = json.load(f)
    except FileNotFoundError:
        all_results = []

    # Append new results
    all_results.append(metrics)

    # Save updated results to file
    with open(save_path, 'w') as f:
        json.dump(all_results, f, indent=4)
    print(f"Results saved successfully with metrics: {metrics}")

In [ ]:
def get_data_generators(data_dir, batch_size, img_size=(224, 224), val_split=0.2):
    """
    Function to create and return training and validation generators.
    """
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        brightness_range=[0.8, 1.2],
        zoom_range=[0.8, 1.2],
        horizontal_flip=True,
        fill_mode='nearest',
        channel_shift_range=20.0,      # Random channel shifts
        vertical_flip=True,            # Flip images vertically
        validation_split=val_split
    )

    train_gen = datagen.flow_from_directory(
        data_dir, target_size=img_size, batch_size=batch_size,
        class_mode='categorical', subset='training'
    )

    val_gen = datagen.flow_from_directory(
        data_dir, target_size=img_size, batch_size=batch_size,
        class_mode='categorical', subset='validation'
    )

    return train_gen, val_gen

In [ ]:
def build_vgg16_model(learning_rate, dropout_rate=None, activation_function='relu', optimizer='adam'):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze all layers

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation=activation_function)(x)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)  # Apply dropout if dropout_rate is specified
    x = Dense(128, activation=activation_function)(x)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)  # Apply dropout if dropout_rate is specified
    output = Dense(4, activation='softmax')(x)

    # Choose optimizer based on parameter
    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        raise ValueError(f"Unsupported optimizer: {optimizer}")

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(
        optimizer=opt,
        loss='categorical_crossentropy',
        metrics=[
            'accuracy',
            AUC(name='auc'),
            Precision(name='precision'),
            Recall(name='recall')
        ]
    )
    return model

In [ ]:
def train_and_evaluate_model(learning_rate, batch_size, dropout_rate, activation_function, optimizer):
    # Load data generators
    train_gen, val_gen = get_data_generators(dataset_dir, batch_size, img_size=(224, 224))

    # Build and compile the model
    model = build_vgg16_model(learning_rate, dropout_rate, activation_function, optimizer)
    early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

    # Train the model and get training history
    history = model.fit(train_gen, validation_data=val_gen, epochs=30, callbacks=[early_stopping])

    # Evaluate on validation data
    val_metrics = model.evaluate(val_gen)

    # Unpack the metrics
    val_loss = val_metrics[0]
    val_accuracy = val_metrics[1]
    val_auc = val_metrics[2]
    val_precision = val_metrics[3]
    val_recall = val_metrics[4]
    val_predictions = model.predict(val_gen)

    # Extract final training metrics from history
    train_loss = history.history['loss'][-1]
    train_accuracy = history.history['accuracy'][-1]

    # Save the model with a unique name based on hyperparameters
    model_name = f"vgg16_lr{learning_rate}_bs{batch_size}_dr{dropout_rate}_{activation_function}_{optimizer}.h5"
    save_dir = os.path.join("/saved_models", model_name)
    model.save(save_dir)
    print(f"Model saved to {save_dir}")

    # Return a dictionary of metrics and model details
    return {
        'train_loss': train_loss,
        'train_accuracy': train_accuracy,
        'val_loss': val_loss,
        'val_accuracy': val_accuracy,
        'val_auc': val_auc,
        'model_name': model_name,
        'hyperparameters': {
            'learning_rate': learning_rate,
            'batch_size': batch_size,
            'dropout_rate': dropout_rate,
            'activation_function': activation_function,
            'optimizer': optimizer
        }
    }

In [ ]:
# Define parameter dictionary with fixed values for each hyperparameter
param_dict = {
    'learning_rate': 0.001,        # Fixed learning rate
    'batch_size': 32,              # Fixed batch size
    'dropout_rate': 0.3,           # Fixed dropout rate
    'activation_function': 'leaky_relu', # Fixed activation function
    'optimizer': 'sgd'            # Fixed optimizer
}

# Example usage: passing `param_dict` to train_and_evaluate_model
metrics = train_and_evaluate_model(**param_dict)

# Print the metrics to verify
print(metrics)

# Save results with parameters and metrics
save_results(metrics)